<a href="https://colab.research.google.com/github/jss265/595R---Deep-Learning-for-Engineers/blob/main/HW1_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Hello Woldies')

Hello Woldies
